In [1]:
## Tropospheric delay correction using GACOS from New Castle U
## for geocoded dataset
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from pysar.objects import timeseries
from pysar.utils import ptime, readfile, utils as ut

ztd_dir = os.path.expanduser('~/insarlab/Kirishima/AlosAT424F620_630/GACOS')
proj_dir = os.path.expanduser('~/insarlab/Kirishima/KirishimaAlosAT424/PYSAR')

out_file = os.path.join(proj_dir, 'INPUTS/GACOS.h5')
ts_file = os.path.join(proj_dir, 'timeseries.h5')
geom_file = os.path.join(proj_dir, 'INPUTS/geometryGeo.h5')

In [2]:
def get_geo_delay(fztd, geo_box, atr, inc_angle):
    meta = readfile.read_attribute(fztd)
    coord = ut.coordinate(meta)
    box = coord.box_geo2pixel(geo_box)
    phs = readfile.read(fztd, box=box)[0]
    phs[phs == 0] = np.nan

    # interpolation
    out_shape = (int(atr['LENGTH']), int(atr['WIDTH']))
    phs = resize(phs, out_shape, order=1, mode='edge', anti_aliasing=True)

    phs /= np.cos(inc_angle * np.pi / 180)
    phs -= phs[int(atr['REF_Y']), int(atr['REF_X'])]
    phs *= -1
    return phs

In [3]:
# 1. get time-series file info
obj = timeseries(ts_file)
obj.open()
atr = readfile.read_attribute(ts_file)

# get box_box
pixel_box = (0, 0, int(atr['WIDTH']), int(atr['LENGTH']))
geo_box = ut.coordinate(atr).box_pixel2geo(pixel_box)

# get inc_angle
inc_angle = readfile.read(geom_file, datasetName='incidenceAngle')[0]

# get grib file list
fztd_list = [os.path.join(ztd_dir, '{}.ztd'.format(i)) for i in obj.dateList]

open timeseries file: timeseries.h5


In [4]:
# calculate deley time-series
num_date = obj.numDate
tropo = np.zeros((num_date, obj.length, obj.width), dtype=np.float32)
prog_bar = ptime.progressBar(maxValue=num_date)
for i in range(num_date):
    fztd = fztd_list[i]
    tropo[i, :, :] = get_geo_delay(fztd, geo_box, atr, inc_angle)
    prog_bar.update(i+1, suffix=os.path.basename(fztd))
prog_bar.close()
tropo -= np.tile(tropo[0, :, :], (num_date, 1, 1))

# write deley to h5 file
obj_out = timeseries(out_file)
obj_out.write2hdf5(data=tropo,
                   dates=obj.dateList,
                   metadata=atr,
                   refFile=ts_file)

[==================================================] 20110407.ztd    3s /     0s 
create timeseries HDF5 file: /Users/yunjunz/insarlab/Kirishima/KirishimaAlosAT424/PYSAR/INPUTS/GACOS.h5 with w mode
create dataset /timeseries of float32    in size of (27, 1441, 2161) with compression=None
create dataset /dates      of |S8        in size of (27,)
create dataset /bperp      of float32    in size of (27,)
finished writing to /Users/yunjunz/insarlab/Kirishima/KirishimaAlosAT424/PYSAR/INPUTS/GACOS.h5


'/Users/yunjunz/insarlab/Kirishima/KirishimaAlosAT424/PYSAR/INPUTS/GACOS.h5'

In [8]:
# correct time-series
work_dir = '/Users/yunjunz/insarlab/Kirishima/KirishimaAlosAT424/PYSAR'
os.chdir(work_dir)
print('Go to directory', work_dir)

!diff.py timeseries.h5 INPUTS/GACOS.h5 -o timeseries_GACOS.h5
!remove_ramp.py timeseries_GACOS.h5 -m maskTempCoh.h5 -s quadratic
!dem_error.py timeseries_GACOS_ramp.h5 -t pysarApp_template.txt

Go to directory /Users/yunjunz/insarlab/Kirishima/KirishimaAlosAT424/PYSAR
timeseries.h5 - ['INPUTS/GACOS.h5'] --> timeseries_GACOS.h5
input files are: timeseries and timeseries
open timeseries file: timeseries.h5
open timeseries file: GACOS.h5
reading timeseries data from file: timeseries.h5 ...
create timeseries HDF5 file: timeseries_GACOS.h5 with w mode
create dataset /timeseries of float32    in size of (27, 1441, 2161) with compression=None
create dataset /dates      of |S8        in size of (27,)
create dataset /bperp      of float32    in size of (27,)
finished writing to timeseries_GACOS.h5
remove quadratic ramp from file: timeseries_GACOS.h5
read mask file: maskTempCoh.h5
reading data ...
estimating phase ramp ...
create timeseries HDF5 file: timeseries_GACOS_ramp.h5 with w mode
create dataset /timeseries of float32    in size of (27, 1441, 2161) with compression=None
create dataset /dates      of |S8        in size of (27,)
create dataset /bperp      of float32    in size of 

In [7]:
!view.py timeseries_GACOS_ramp_demErr.h5 -v -5 5 --nodisplay
!view.py timeseries_ECMWF_ramp_demErr.h5 -v -5 5 --nodisplay

run view.py in PySAR version 1.0.0-dev, release date 2019-03-31
input file is timeseries file: /Users/yunjunz/insarlab/Kirishima/KirishimaAlosAT424/PYSAR/timeseries_GACOS_ramp_demErr.h5 in float32 format
file size in y/x: (1441, 2161)
num of datasets in file timeseries_GACOS_ramp_demErr.h5: 27
datasets to exclude (0):
[]
datasets to display (27):
['timeseries-20060924', 'timeseries-20061225', 'timeseries-20070627', 'timeseries-20070812', 'timeseries-20070927', 'timeseries-20071112', 'timeseries-20071228', 'timeseries-20080212', 'timeseries-20080329', 'timeseries-20080514', 'timeseries-20080629', 'timeseries-20080929', 'timeseries-20081114', 'timeseries-20081230', 'timeseries-20090214', 'timeseries-20090702', 'timeseries-20090817', 'timeseries-20091002', 'timeseries-20100102', 'timeseries-20100217', 'timeseries-20100404', 'timeseries-20100520', 'timeseries-20100705', 'timeseries-20100820', 'timeseries-20101120', 'timeseries-20110220', 'timeseries-20110407']
data   coverage in y/x: (0, 0